# Analyse der PV-Anlagen in der Schweiz
Dieses Notebook analysiert die Photovoltaik-Anlagen (PV-Anlagen) in der Schweiz, einschliesslich ihrer Verteilung und Entwicklung.

## Daten laden, aufbereiten und anreichern
In diesem Abschnitt werden die Daten geladen, gefiltert und für die Analyse vorbereitet.

In [2]:
# Bibliotheken importieren
import pandas as pd

# CSV-Datei mit den PV-Anlagen laden
file_path = '../data/electricity_production_plants/ElectricityProductionPlant.csv'
df = pd.read_csv(file_path)

# Kategorien ersetzen (Haupt-, Unter- und Anlagenkategorien)
main_categories = pd.read_csv('../data/electricity_production_plants/MainCategoryCatalogue.csv')
df['MainCategory'] = df['MainCategory'].replace(dict(zip(main_categories['Catalogue_id'], main_categories['de'])))

sub_categories = pd.read_csv('../data/electricity_production_plants/SubCategoryCatalogue.csv')
df['SubCategory'] = df['SubCategory'].replace(dict(zip(sub_categories['Catalogue_id'], sub_categories['de'])))

plant_categories = pd.read_csv('../data/electricity_production_plants/PlantCategoryCatalogue.csv')
df['PlantCategory'] = df['PlantCategory'].replace(dict(zip(plant_categories['Catalogue_id'], plant_categories['de'])))

# Nur Photovoltaik-Anlagen filtern
df = df[df['SubCategory'] == 'Photovoltaik'].copy()

# Jahr aus 'BeginningOfOperation' extrahieren
df['Year'] = pd.to_datetime(df['BeginningOfOperation'], errors='coerce').dt.year

# Erste Zeilen anzeigen
df.head()

,xtf_id,Address,PostCode,Municipality,Canton,BeginningOfOperation,InitialPower,TotalPower,MainCategory,SubCategory,PlantCategory,_x,_y,Year
0,14727,Schlossstrasse 15,4147,Aesch BL,BL,2009-05-05,14.65,18.81,Übrige erneuerbare Energien,Photovoltaik,Integriert,2611936.0,1257011.0,2009
1,14728,Ob der Steig 1,5082,Kaisten,AG,2011-10-28,5.80,5.80,Übrige erneuerbare Energien,Photovoltaik,Angebaut,2645758.0,1265094.0,2011
2,10164,Gässli 4,3114,Wichtrach,BE,2008-10-07,3.00,3.00,Übrige erneuerbare Energien,Photovoltaik,Angebaut,2610547.0,1188979.0,2008
3,10794,Diemtigtalstrasse 46,3753,Oey,BE,2008-06-27,8.40,8.40,Übrige erneuerbare Energien,Photovoltaik,Angebaut,2610529.0,1167346.0,2008
4,9476,Holzweidstrasse 8,8340,Hinwil,ZH,2006-04-21,4.80,4.80,Übrige erneuerbare Energien,Photovoltaik,Angebaut,2705863.0,1240553.0,2006


## Karte mit allen PV-Anlagen in der Schweiz (Stand 2024)
Diese Karte zeigt die Standorte aller PV-Anlagen in der Schweiz bis zum Jahr 2024.

In [ ]:
# Bibliotheken für die Kartenerstellung importieren
import folium
from folium.plugins import FastMarkerCluster
from pyproj import Transformer

# Daten für Installationen bis 2024 filtern
df['BeginningOfOperation'] = pd.to_datetime(df['BeginningOfOperation'], errors='coerce')
df = df[df['BeginningOfOperation'].dt.year <= 2024]

# Karte erstellen
map = folium.Map(location=[46.8, 8.33], zoom_start=7, tiles='OpenStreetMap')

# Marker-Cluster hinzufügen
transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)
coordinates = [
    [transformer.transform(row['_x'], row['_y'])[1], transformer.transform(row['_x'], row['_y'])[0]]
    for _, row in df.iterrows() if not pd.isna(row['_x']) and not pd.isna(row['_y'])
]
FastMarkerCluster(coordinates).add_to(map)

# Titel hinzufügen
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>PV-Anlagen in der Schweiz (Stand 2024)</b></div>
'''
map.get_root().html.add_child(folium.Element(title_html))

# Karte speichern und anzeigen
map.save('../docs/assets/diagramme/elcoms_map.html')
map

## Kumulative Entwicklung der Gesamtkapazität
Dieses Diagramm zeigt die kumulative Entwicklung der Gesamtkapazität der PV-Anlagen pro Kanton

In [ ]:
# Plotly für die Visualisierung importieren
import plotly.express as px

# Daten gruppieren und kumulativ berechnen
cumulative_data = df.groupby(['Year', 'Canton'])['TotalPower'].sum().groupby(level=1).cumsum().reset_index()
filtered_data = cumulative_data[(cumulative_data['Year'] >= 2015) & (cumulative_data['Year'] <= 2024)].copy()
filtered_data['CumulativePower'] = cumulative_data.groupby('Canton')['TotalPower'].cumsum()

# Farben für die Top 4 Kantone und neutrale Farbe für die restlichen
top_4_cantons = ['BE', 'ZH', 'VD', 'SG']
filtered_data['Color'] = filtered_data['Canton'].apply(
    lambda x: (
        'rgb(0, 102, 204)' if x == top_4_cantons[0] else
        'rgb(0, 153, 76)' if x == top_4_cantons[1] else
        'rgb(255, 153, 51)' if x == top_4_cantons[2] else
        'rgb(204, 0, 0)' if x == top_4_cantons[3] else
        'rgb(169, 169, 169)'
    )
)

# Liniendiagramm erstellen
fig = px.line(
    filtered_data,
    x='Year',
    y='CumulativePower',
    color='Canton',
    title='Kumulative Entwicklung der Gesamtkapazität pro Kanton (MW)',
    labels={'CumulativePower': 'Kumulative Gesamtkapazität (MW)', 'Canton': 'Kanton'},
    color_discrete_map={canton: color for canton, color in zip(filtered_data['Canton'], filtered_data['Color'])}
)
fig.update_traces(
    line=dict(width=2),
    selector=dict(name=lambda name: name not in top_4_cantons),
    line_color='rgb(169, 169, 169)'
)
fig.update_layout(
    xaxis=dict(tickmode='linear', tick0=2015, dtick=1, title=''),
    template='simple_white',
    showlegend=False  # Remove the legend
)

# Add canton labels at the end of the lines for the top 4 cantons
for canton in top_4_cantons:
    canton_data = filtered_data[filtered_data['Canton'] == canton]
    if not canton_data.empty:
        last_year = canton_data['Year'].max()
        last_value = canton_data[canton_data['Year'] == last_year]['CumulativePower'].values[0]
        fig.add_annotation(
            x=last_year,
            y=last_value,
            text=canton,
            showarrow=False,
            font=dict(size=10),
            xanchor='left'
        )
fig.show()
fig.write_html("../docs/assets/diagramme/elcom_entwicklung_total_power.html")

## Gesamtkapazität pro Kanton im Jahr 2024
Dieses Balkendiagramm zeigt die Gesamtkapazität der PV-Anlagen pro Kanton im Jahr 2024.

In [6]:
# Auf Jahre 2015–2024 beschränken
df = df[df['Year'].between(2015, 2024)]

# Kumulative Gesamtkapazität pro Kanton und Jahr berechnen
df_cumulative = (
    df.groupby(['Year', 'Canton'])['TotalPower']
    .sum()
    .groupby(level=1).cumsum()
    .reset_index()
)

# Farben zuweisen: Top 4 Kantone pro Jahr individuell einfärben
def assign_color(row, top4):
    return (
        'rgb(0, 102, 204)' if row['Canton'] == top4[0] else  # Blau
        'rgb(0, 153, 76)' if row['Canton'] == top4[1] else  # Grün
        'rgb(255, 153, 51)' if row['Canton'] == top4[2] else  # Orange
        'rgb(204, 0, 0)' if row['Canton'] == top4[3] else  # Rot
        'rgb(169, 169, 169)'  # Grau
    )

colored_data = []
for year in range(2015, 2025):
    year_data = df_cumulative[df_cumulative['Year'] == year].copy()
    top4 = year_data.sort_values(by='TotalPower', ascending=False).head(4)['Canton'].tolist()
    year_data['Color'] = year_data.apply(lambda row: assign_color(row, top4), axis=1)
    colored_data.append(year_data)

df_colored = pd.concat(colored_data)

# Maximalwert für Y-Achse mit 10% Puffer
max_y = df_colored['TotalPower'].max() * 1.1

import plotly.express as px

fig = px.bar(
    df_colored,
    x='Canton',
    y='TotalPower',
    animation_frame='Year',
    title='Kumulierte Gesamtkapazität (Photovoltaik) pro Kanton (2015–2024)',
    labels={'TotalPower': 'Kumulierte Gesamtkapazität (MW)'},
    color='Color',
    color_discrete_map='identity'
)

fig.update_traces(text=None)

fig.update_layout(
    xaxis=dict(categoryorder='total descending', title='Kanton'),
    yaxis=dict(range=[0, max_y], title='Kumulierte Gesamtkapazität (MW)'),
    template='simple_white',
    showlegend=False
)

# Slider mit redraw, damit Y-Achse sich dynamisch anpasst
fig.layout.sliders[0]['steps'] = [
    dict(
        args=[[str(year)], {"frame": {"duration": 500, "redraw": True}, "mode": "immediate"}],
        label=str(year),
        method="animate"
    )
    for year in df_colored['Year'].unique()
]

fig.write_html("../docs/assets/diagramme/elcom_total_power_2015_2024_kumuliert.html")

fig.show()


## Karte mit kumulierter Gesamtkapazität der PV-Anlagen pro Gemeinde bis 2024
Diese Karte zeigt die kumulierte Gesamtkapazität der PV-Anlagen in jeder Gemeinde bis zum Jahr 2024.

In [ ]:
import folium
import json
from shapely.geometry import shape, mapping
from shapely.ops import transform
from functools import partial
import pyproj

# Load CSV File
file_path = '../data/electricity_production_plants/AMTOVZ_CSV_WGS84.csv'
df_bfs_to_plz = pd.read_csv(file_path, delimiter=';')

# Merge DataFrames
merged_df = df.merge(
    df_bfs_to_plz[['PLZ', 'BFS-Nr']],
    left_on='PostCode',
    right_on='PLZ',
    how='left',
    indicator=True
)

# Prepare cumulative data
cumulative_map_data = merged_df[merged_df['Year'] <= 2024].groupby(['BFS-Nr'])['TotalPower'].sum().reset_index()

# Load and simplify GeoJSON
with open('../data/electricity_production_plants/gemeinden.geojson', 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Assign default values to missing municipalities
average_power = cumulative_map_data['TotalPower'].mean()
missing_bfs = set(geojson_data['features'][i]['properties']['gemeinde_BFS_NUMMER'] for i in range(len(geojson_data['features']))) - set(cumulative_map_data['BFS-Nr'])
for bfs in missing_bfs:
    cumulative_map_data = pd.concat([cumulative_map_data, pd.DataFrame({'BFS-Nr': [bfs], 'TotalPower': [average_power]})], ignore_index=True)

simplified_features = []
for feature in geojson_data['features']:
    geom = shape(feature['geometry'])
    simplify = partial(
        pyproj.Transformer.from_crs('EPSG:4326', 'EPSG:4326', always_xy=True).transform
    )
    simplified_geom = transform(simplify, geom).simplify(0.001, preserve_topology=True)
    feature['geometry'] = mapping(simplified_geom)
    bfs_nr = feature['properties']['gemeinde_BFS_NUMMER']
    total_power = cumulative_map_data.loc[cumulative_map_data['BFS-Nr'] == bfs_nr, 'TotalPower'].values[0] if bfs_nr in cumulative_map_data['BFS-Nr'].values else 0
    feature['properties']['TotalPower'] = f"{int(total_power):,}".replace(",", "'")
    feature['properties']['bfs-str'] = str(feature['properties']['gemeinde_BFS_NUMMER'])
    simplified_features.append(feature)

geojson_data['features'] = simplified_features

map = folium.Map(location=[46.8, 8.33], zoom_start=7)

# Add choropleth layer
folium.Choropleth(
    geo_data=geojson_data,
    name='Choropleth',
    data=cumulative_map_data,
    columns=['BFS-Nr', 'TotalPower'],
    key_on='feature.properties.gemeinde_BFS_NUMMER',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Total Power (MW)'
).add_to(map)

# Add tooltips and customize boundary style
folium.GeoJson(
    geojson_data,
    name="Gemeinden",
    style_function=lambda feature: {
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['gemeinde_NAME', 'bfs-str', 'TotalPower'],
        aliases=['Gemeinde:', 'BFS-Nummer:', 'Gesamtkapazität (MW):'],
        localize=True
    )
).add_to(map)

# Add title and save map
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>Choroplethenkarte von der Leistung der PV Anlagen pro Gemeinde</b></div>
'''
map.get_root().html.add_child(folium.Element(title_html))

map.save('../docs/assets/diagramme/map_cumulative_pv_power.html')

map